# Final MLP for Category prediction

In [1]:
import numpy as np
import pandas as pd

In [2]:
import gzip
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [3]:
import shutil
with open('train.json', 'rb') as f_in:
    with gzip.open('train.json.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [4]:
import shutil
with open('test_Category.json', 'rb') as f_in:
    with gzip.open('test_Category.json.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

## convert the json to csv for later use

##### Training

In [5]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('train.json.gz')

In [6]:
df

,reviewTime,reviewText,helpful,reviewerID,reviewHash,categories,unixReviewTime,itemID,rating,summary,categoryID,price
0,"09 26, 2013",The model in this picture has them rolled up a...,"{'nHelpful': 0, 'outOf': 0}",U490934656,R798569390,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",1380153600,I402344648,4.0,High Waisted,0,NaN
1,"02 7, 2013","I love the look of this bra, it is what I want...","{'nHelpful': 0, 'outOf': 0}",U714157797,R436443063,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",1360195200,I697650540,4.0,Beautiful but size runs small,0,NaN
2,"03 16, 2014",I am not comfortable with wearing my wedding b...,"{'nHelpful': 0, 'outOf': 0}",U507366950,R103439446,"[[Clothing, Shoes & Jewelry, Wedding Party Gif...",1394928000,I464613034,5.0,Great Alternative for Nurses,0,19.99
3,"03 10, 2014",Like the look of this top and really looks cut...,"{'nHelpful': 0, 'outOf': 0}",U307862152,R486351639,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",1394409600,I559560885,2.0,One size fits all...Questionable,0,18.99
4,"07 30, 2013",I'm quite small and the XS fits me like a regu...,"{'nHelpful': 1, 'outOf': 1}",U742726598,R508664275,"[[Clothing, Shoes & Jewelry, Women, Plus-Size,...",1375142400,I476005312,5.0,Great shirt,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
199995,"01 26, 2013",Looks just like the picture and the item arriv...,"{'nHelpful': 0, 'outOf': 0}",U781794983,R285432298,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",1359158400,I245323432,5.0,Cute,0,NaN
199996,"04 2, 2013",I'm a C cup and the C cup in this is way too s...,"{'nHelpful': 1, 'outOf': 1}",U151975942,R454986738,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",1364860800,I990230316,1.0,Not true to Cup size.,0,NaN
199997,"09 22, 2012",I have a pair of earrings just like this (only...,"{'nHelpful': 0, 'outOf': 0}",U525354881,R088851171,"[[Clothing, Shoes & Jewelry, Jewelry: Internat...",1348272000,I037381245,4.0,"Lovely, but small",0,34.99
199998,"12 27, 2013",My granddaughter love the scarf and fit perfec...,"{'nHelpful': 0, 'outOf': 1}",U995566285,R524991477,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",1388102400,I343675670,5.0,Scarf,0,7.50


In [7]:
#catDict = {
#  "Women": 0,
#  "Men": 1,
#  "Girls": 2,
#  "Boys": 3,
#  "Baby": 4
#}

df.categoryID.value_counts()

0    141398
1     51416
4      2976
2      2329
3      1881
Name: categoryID, dtype: int64

In [8]:
import string

input_data = df
input_data['category'] = input_data['categoryID'].astype(object) # fix datatype error
input_data['reviewText'] = input_data['reviewText'].astype(object) # fix datatype error

dataset = {"reviewText": input_data["reviewText"], "category": input_data["category"]  }
dataset = pd.DataFrame(data = dataset)
dataset = dataset.dropna()

# dataset = dataset[dataset["overall"] != '3'] # need datatype=object
dataset["label"] = dataset["category"]

In [9]:
dataset

,reviewText,category,label
0,The model in this picture has them rolled up a...,0,0
1,"I love the look of this bra, it is what I want...",0,0
2,I am not comfortable with wearing my wedding b...,0,0
3,Like the look of this top and really looks cut...,0,0
4,I'm quite small and the XS fits me like a regu...,0,0
...,...,...,...
199995,Looks just like the picture and the item arriv...,0,0
199996,I'm a C cup and the C cup in this is way too s...,0,0
199997,I have a pair of earrings just like this (only...,0,0
199998,My granddaughter love the scarf and fit perfec...,0,0


## Create Train and Test

In [10]:
test = getDF('test_Category.json.gz')

In [11]:
test

,reviewTime,reviewText,helpful,reviewerID,reviewHash,unixReviewTime,rating,summary,price
0,"07 26, 2013","I love this blouse, in fact I have it on right...","{'nHelpful': 9, 'outOf': 9}",U281659737,R934811302,1374796800,5.0,love it,NaN
1,"06 8, 2014",Cute product. Loved the fit. Fast shipping! I ...,"{'nHelpful': 0, 'outOf': 0}",U670561057,R657711680,1402185600,4.0,Cute but a bit uncomfortable,NaN
2,"03 21, 2012",I wanted a formal watch that had a big face an...,"{'nHelpful': 0, 'outOf': 0}",U433746872,R750304163,1332288000,5.0,"Bold, Large-face Watch",NaN
3,"06 27, 2014",My daughter used this dress for her first comm...,"{'nHelpful': 0, 'outOf': 0}",U327816997,R865011815,1403827200,5.0,The perfect Dress,NaN
4,"03 21, 2014","Nice shirt, good quality with pockets. Kind o...","{'nHelpful': 1, 'outOf': 1}",U323131234,R222729968,1395360000,5.0,Nice shirt,NaN
...,...,...,...,...,...,...,...,...,...
13995,"12 6, 2012",The results of this outfit have been . . . wel...,"{'nHelpful': 0, 'outOf': 0}",U648420317,R534448754,1354752000,5.0,Very sexy,15.98
13996,"10 17, 2013",These are my &#34;GO-TO&#34; bras and the only...,"{'nHelpful': 0, 'outOf': 0}",U807796696,R893942246,1381968000,5.0,Bras for Life!,NaN
13997,"01 17, 2012",I walk to work. In NYC. For half an hour each ...,"{'nHelpful': 1, 'outOf': 2}",U817651487,R553584002,1326758400,3.0,Warm but...,NaN
13998,"01 11, 2013",I love this shirt. It's so comfortable and eve...,"{'nHelpful': 0, 'outOf': 0}",U372716165,R669751265,1357862400,5.0,Great top,35.00


In [12]:
train = dataset

In [13]:
train

,reviewText,category,label
0,The model in this picture has them rolled up a...,0,0
1,"I love the look of this bra, it is what I want...",0,0
2,I am not comfortable with wearing my wedding b...,0,0
3,Like the look of this top and really looks cut...,0,0
4,I'm quite small and the XS fits me like a regu...,0,0
...,...,...,...
199995,Looks just like the picture and the item arriv...,0,0
199996,I'm a C cup and the C cup in this is way too s...,0,0
199997,I have a pair of earrings just like this (only...,0,0
199998,My granddaughter love the scarf and fit perfec...,0,0


In [14]:
train_X = train["reviewText"]
test_X = test["reviewText"]
train_y = train["label"]
train_y = train_y.astype('int')

In [15]:
train_X

0         The model in this picture has them rolled up a...
1         I love the look of this bra, it is what I want...
2         I am not comfortable with wearing my wedding b...
3         Like the look of this top and really looks cut...
4         I'm quite small and the XS fits me like a regu...
                                ...                        
199995    Looks just like the picture and the item arriv...
199996    I'm a C cup and the C cup in this is way too s...
199997    I have a pair of earrings just like this (only...
199998    My granddaughter love the scarf and fit perfec...
199999    My baby wore it only one time and there are ho...
Name: reviewText, Length: 200000, dtype: object

# Category Prediction Baseline

In [16]:
### Category prediction baseline: Just consider some of the most common words from each category

catDict = {
  "Women": 0,
  "Men": 1,
  "Girls": 2,
  "Boys": 3,
  "Baby": 4
}

predictions = open("predictions_Category.txt", 'w')
predictions.write("reviewerID-reviewHash,category\n")
for l in readGz("test_Category.json.gz"):
  cat = catDict['Women'] # If there's no evidence, just choose the most common category in the dataset
  words = l['reviewText'].lower()
  if 'wife' in words:
    cat = catDict['Women']
  if 'husband' in words:
    cat = catDict['Men']
  if 'daughter' in words:
    cat = catDict['Girls']
  if 'son' in words:
    cat = catDict['Boys']
  if 'baby' in words:
    cat = catDict['Baby']
  predictions.write(l['reviewerID'] + '-' + l['reviewHash'] + "," + str(cat) + "\n")

predictions.close()


## Word Vectorization

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b', ngram_range=(1, 1), max_features = 230000) #, max_df = 0.9)
train_vector = vectorizer.fit_transform(train["reviewText"])
test_vector = vectorizer.transform(test["reviewText"])

In [18]:
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=230000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='\\b\\w+\\b', tokenizer=None,
                vocabulary=None)

In [19]:
train["reviewText"]

0         The model in this picture has them rolled up a...
1         I love the look of this bra, it is what I want...
2         I am not comfortable with wearing my wedding b...
3         Like the look of this top and really looks cut...
4         I'm quite small and the XS fits me like a regu...
                                ...                        
199995    Looks just like the picture and the item arriv...
199996    I'm a C cup and the C cup in this is way too s...
199997    I have a pair of earrings just like this (only...
199998    My granddaughter love the scarf and fit perfec...
199999    My baby wore it only one time and there are ho...
Name: reviewText, Length: 200000, dtype: object

In [20]:
train_vector

<200000x64008 sparse matrix of type '<class 'numpy.int64'>'
	with 8823470 stored elements in Compressed Sparse Row format>

### now build the model

In [22]:
train_y

0         0
1         0
2         0
3         0
4         0
         ..
199995    0
199996    0
199997    0
199998    0
199999    4
Name: label, Length: 200000, dtype: int32

# Multi Layer Perceptron

In [37]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='sgd', alpha=0.00001, learning_rate = 'adaptive', # 
                     hidden_layer_sizes=(25,), random_state=1) # best: solver='adam', hidden_layer_sizes=(50,25,10,5)

clf.fit(train_vector, train_y.values.ravel())

C:\Users\joshd\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(25,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='sgd', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [38]:
ynew = clf.predict(test_vector)

In [39]:
ynew

array([0, 0, 1, ..., 0, 0, 0])

In [40]:
type(ynew)

numpy.ndarray

### add those predictions to file for kaggle evaluation

In [41]:
test

,reviewTime,reviewText,helpful,reviewerID,reviewHash,unixReviewTime,rating,summary,price,category,reviewerID-reviewHash
0,"07 26, 2013","I love this blouse, in fact I have it on right...","{'nHelpful': 9, 'outOf': 9}",U281659737,R934811302,1374796800,5.0,love it,NaN,0,U281659737-R934811302
1,"06 8, 2014",Cute product. Loved the fit. Fast shipping! I ...,"{'nHelpful': 0, 'outOf': 0}",U670561057,R657711680,1402185600,4.0,Cute but a bit uncomfortable,NaN,0,U670561057-R657711680
2,"03 21, 2012",I wanted a formal watch that had a big face an...,"{'nHelpful': 0, 'outOf': 0}",U433746872,R750304163,1332288000,5.0,"Bold, Large-face Watch",NaN,1,U433746872-R750304163
3,"06 27, 2014",My daughter used this dress for her first comm...,"{'nHelpful': 0, 'outOf': 0}",U327816997,R865011815,1403827200,5.0,The perfect Dress,NaN,0,U327816997-R865011815
4,"03 21, 2014","Nice shirt, good quality with pockets. Kind o...","{'nHelpful': 1, 'outOf': 1}",U323131234,R222729968,1395360000,5.0,Nice shirt,NaN,1,U323131234-R222729968
...,...,...,...,...,...,...,...,...,...,...,...
13995,"12 6, 2012",The results of this outfit have been . . . wel...,"{'nHelpful': 0, 'outOf': 0}",U648420317,R534448754,1354752000,5.0,Very sexy,15.98,0,U648420317-R534448754
13996,"10 17, 2013",These are my &#34;GO-TO&#34; bras and the only...,"{'nHelpful': 0, 'outOf': 0}",U807796696,R893942246,1381968000,5.0,Bras for Life!,NaN,0,U807796696-R893942246
13997,"01 17, 2012",I walk to work. In NYC. For half an hour each ...,"{'nHelpful': 1, 'outOf': 2}",U817651487,R553584002,1326758400,3.0,Warm but...,NaN,1,U817651487-R553584002
13998,"01 11, 2013",I love this shirt. It's so comfortable and eve...,"{'nHelpful': 0, 'outOf': 0}",U372716165,R669751265,1357862400,5.0,Great top,35.00,0,U372716165-R669751265


In [42]:
test['category'] = ynew # str(ynew)

In [43]:
test

,reviewTime,reviewText,helpful,reviewerID,reviewHash,unixReviewTime,rating,summary,price,category,reviewerID-reviewHash
0,"07 26, 2013","I love this blouse, in fact I have it on right...","{'nHelpful': 9, 'outOf': 9}",U281659737,R934811302,1374796800,5.0,love it,NaN,0,U281659737-R934811302
1,"06 8, 2014",Cute product. Loved the fit. Fast shipping! I ...,"{'nHelpful': 0, 'outOf': 0}",U670561057,R657711680,1402185600,4.0,Cute but a bit uncomfortable,NaN,0,U670561057-R657711680
2,"03 21, 2012",I wanted a formal watch that had a big face an...,"{'nHelpful': 0, 'outOf': 0}",U433746872,R750304163,1332288000,5.0,"Bold, Large-face Watch",NaN,1,U433746872-R750304163
3,"06 27, 2014",My daughter used this dress for her first comm...,"{'nHelpful': 0, 'outOf': 0}",U327816997,R865011815,1403827200,5.0,The perfect Dress,NaN,0,U327816997-R865011815
4,"03 21, 2014","Nice shirt, good quality with pockets. Kind o...","{'nHelpful': 1, 'outOf': 1}",U323131234,R222729968,1395360000,5.0,Nice shirt,NaN,1,U323131234-R222729968
...,...,...,...,...,...,...,...,...,...,...,...
13995,"12 6, 2012",The results of this outfit have been . . . wel...,"{'nHelpful': 0, 'outOf': 0}",U648420317,R534448754,1354752000,5.0,Very sexy,15.98,0,U648420317-R534448754
13996,"10 17, 2013",These are my &#34;GO-TO&#34; bras and the only...,"{'nHelpful': 0, 'outOf': 0}",U807796696,R893942246,1381968000,5.0,Bras for Life!,NaN,0,U807796696-R893942246
13997,"01 17, 2012",I walk to work. In NYC. For half an hour each ...,"{'nHelpful': 1, 'outOf': 2}",U817651487,R553584002,1326758400,3.0,Warm but...,NaN,0,U817651487-R553584002
13998,"01 11, 2013",I love this shirt. It's so comfortable and eve...,"{'nHelpful': 0, 'outOf': 0}",U372716165,R669751265,1357862400,5.0,Great top,35.00,0,U372716165-R669751265


In [44]:
test["reviewerID-reviewHash"] = test["reviewerID"].str.cat(test[["reviewHash"]].astype(str), sep="-")

In [45]:
test

,reviewTime,reviewText,helpful,reviewerID,reviewHash,unixReviewTime,rating,summary,price,category,reviewerID-reviewHash
0,"07 26, 2013","I love this blouse, in fact I have it on right...","{'nHelpful': 9, 'outOf': 9}",U281659737,R934811302,1374796800,5.0,love it,NaN,0,U281659737-R934811302
1,"06 8, 2014",Cute product. Loved the fit. Fast shipping! I ...,"{'nHelpful': 0, 'outOf': 0}",U670561057,R657711680,1402185600,4.0,Cute but a bit uncomfortable,NaN,0,U670561057-R657711680
2,"03 21, 2012",I wanted a formal watch that had a big face an...,"{'nHelpful': 0, 'outOf': 0}",U433746872,R750304163,1332288000,5.0,"Bold, Large-face Watch",NaN,1,U433746872-R750304163
3,"06 27, 2014",My daughter used this dress for her first comm...,"{'nHelpful': 0, 'outOf': 0}",U327816997,R865011815,1403827200,5.0,The perfect Dress,NaN,0,U327816997-R865011815
4,"03 21, 2014","Nice shirt, good quality with pockets. Kind o...","{'nHelpful': 1, 'outOf': 1}",U323131234,R222729968,1395360000,5.0,Nice shirt,NaN,1,U323131234-R222729968
...,...,...,...,...,...,...,...,...,...,...,...
13995,"12 6, 2012",The results of this outfit have been . . . wel...,"{'nHelpful': 0, 'outOf': 0}",U648420317,R534448754,1354752000,5.0,Very sexy,15.98,0,U648420317-R534448754
13996,"10 17, 2013",These are my &#34;GO-TO&#34; bras and the only...,"{'nHelpful': 0, 'outOf': 0}",U807796696,R893942246,1381968000,5.0,Bras for Life!,NaN,0,U807796696-R893942246
13997,"01 17, 2012",I walk to work. In NYC. For half an hour each ...,"{'nHelpful': 1, 'outOf': 2}",U817651487,R553584002,1326758400,3.0,Warm but...,NaN,0,U817651487-R553584002
13998,"01 11, 2013",I love this shirt. It's so comfortable and eve...,"{'nHelpful': 0, 'outOf': 0}",U372716165,R669751265,1357862400,5.0,Great top,35.00,0,U372716165-R669751265


In [46]:
output_cat_0614_MLP = test[['reviewerID-reviewHash','category']]

In [47]:
output_cat_0614_MLP

,reviewerID-reviewHash,category
0,U281659737-R934811302,0
1,U670561057-R657711680,0
2,U433746872-R750304163,1
3,U327816997-R865011815,0
4,U323131234-R222729968,1
...,...,...
13995,U648420317-R534448754,0
13996,U807796696-R893942246,0
13997,U817651487-R553584002,0
13998,U372716165-R669751265,0


In [48]:
output_cat_0614_MLP.category.value_counts()

0    10367
1     3284
4      191
3       89
2       69
Name: category, dtype: int64

### change data on txt file below

In [49]:
# output_cat_0614_MLP.to_csv('output_cat_0614_MLP2_25.txt', header=True, index=False)